In [2]:
from tokenize import String

"""시뮬레이션 결과파일에서 데이터 가져오기"""
import sqlite3
import pandas as pd

# 데이터베이스 파일 경로
db_path = "C:/VISSIM_Workspace/test/test.results/3.db"  # 실제 경로로 변경하세요
# 파일명 추출
simrun = db_path.split("/")[-1].split(".")[0]

# 데이터베이스 연결
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 특정 테이블의 데이터 조회
table_name = "DATACOLLECTIONMEASUREMENT_EvaluationTimeIntervalClass"
query = f"SELECT * FROM {table_name};"  # 전체 데이터 조회

# 데이터 읽기
df = pd.read_sql_query(query, conn)

# TIMEINT 변환
df["ARG_TIMEINTERVAL"] = df["ARG_TIMEINTERVAL"].astype(int) * 60 - 60  # 변환 시작점 보정
df["ARG_TIMEINTERVAL"] = df["ARG_TIMEINTERVAL"].apply(lambda x: f"{x}-{x+60}")

# OCCUPRATE 보정
df["OCCUPRATE"] = df["OCCUPRATE"].astype(float)*100
df["OCCUPRATE"] = df["OCCUPRATE"].apply(lambda x: f"{x:.2f}")  # 항상 소수점 둘째 자리까지 유지
df = df.round(2)
df["OCCUPRATE"] = df["OCCUPRATE"].astype(str) + "%"
print(df["OCCUPRATE"].head())

# SIMRUN 설정
df["SIMRUN"] =simrun

# 컬럼명 변경
df.rename(columns={
    "ARG_TIMEINTERVAL": "TIMEINT",
    "OBJECT_ID": "DATACOLLECTIONMEASUREMENT",
    "ACCELERATION" : "ACCELERATION(ALL)",
    "DIST" : "DIST(ALL)",
    "LENGTH" : "LENGTH(ALL)",
    "VEHS" : "VEHS(ALL)",
    "PERS" : "PERS(ALL)",
    "QUEUEDELAY" : "QUEUEDELAY(ALL)",
    "SPEEDAVGARITH" : "SPEEDAVGARITH(ALL)",
    "SPEEDAVGHARM" : "SPEEDAVGHARM(ALL)",
    "OCCUPRATE" : "OCCUPRATE(ALL)"
}, inplace=True)

# 순서 재정의
df = df[["SIMRUN", "TIMEINT", "DATACOLLECTIONMEASUREMENT", "ACCELERATION(ALL)", "DIST(ALL)",
         "LENGTH(ALL)", "VEHS(ALL)", "PERS(ALL)", "QUEUEDELAY(ALL)", "SPEEDAVGARITH(ALL)", "SPEEDAVGHARM(ALL)", "OCCUPRATE(ALL)"]]
print(df)

df.to_excel("C:\\VISSIM_Workspace\\test\\output.xlsx", index=False)


# 연결 종료
conn.close()


0    0.68%
1    0.68%
2    0.68%
3    0.50%
4    0.50%
Name: OCCUPRATE, dtype: object
    SIMRUN    TIMEINT DATACOLLECTIONMEASUREMENT  ACCELERATION(ALL)  DIST(ALL)  \
0        3       0-60                         1               0.16      34.25   
1        3       0-60                         2               0.16      43.66   
2        3       0-60                         3               0.16      54.10   
3        3     60-120                         1               0.20      35.28   
4        3     60-120                         2               0.20      44.19   
..     ...        ...                       ...                ...        ...   
175      3  3480-3540                         2               0.20      44.88   
176      3  3480-3540                         3               0.20      55.20   
177      3  3540-3600                         1               0.18      33.92   
178      3  3540-3600                         2               0.18      43.62   
179      3  3540-3600  